### A fresh start from old code

##### we will use this as a templater for futrue, won't modify this at all

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
from sklearn import preprocessing as pp
from sklearn import cross_validation as cv
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from sklearn.svm import SVC
from sklearn import tree
from sklearn.cross_validation import cross_val_score

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### My helper funcions 

In [29]:
def get_final_output(clf, X, y, X_test, out_file):
    clf.fit(X, y)
    y_ = clf.predict(X_test)
    y_results = pd.DataFrame()
    
    y_results["PassengerId"] = X_test.index
    y_results["Survived"] = y_
    
    if(out_file):
        y_results.to_csv(out_file, encoding='utf-8', index=False)

In [4]:
def report_train_accuracy(clf, X, y):
    clf.fit(X, y)
    y_pred = clf.predict(X)
    
    #Confusion Matrix
    y_test = y
    confusion_mat = metrics.confusion_matrix(y_test, y_pred)
    
    TN = confusion_mat[0, 0]
    TP = confusion_mat[1, 1]
    FP = confusion_mat[0, 1]
    FN = confusion_mat[1, 0]
    
    
    print("Confusion Matrix : ")
    print(confusion_mat)
 
    print("Sensitivity :")
    print(metrics.recall_score(y_test, y_pred))
    
    print("Precision : ")
    print(TP/float(TP+FP))
    
    print("Specificity : ")
    print(TN/float(TN+FP))
    
    print("False Positive Rate : ")
    print(FP/float(TN+FP)) 
    
    print("Training Accuracy : ", metrics.accuracy_score(y, y_pred))
    return metrics.accuracy_score(y, y_pred)

In [5]:
def report_train_test_accuracy(clf, X, y, test_size, random_state):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test_size, random_state = random_state)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    #Confusion Matrix
    confusion_mat = metrics.confusion_matrix(y_test, y_pred)
    
    TN = confusion_mat[0, 0]
    TP = confusion_mat[1, 1]
    FP = confusion_mat[0, 1]
    FN = confusion_mat[1, 0]
    
    
    print("Confusion Matrix : ")
    print(confusion_mat)
 
    print("Sensitivity :")
    print(metrics.recall_score(y_test, y_pred))
    
    print("Precision : ")
    print(TP/float(TP+FP))
    
    print("Specificity : ")
    print(TN/float(TN+FP))
    
    print("False Positive Rate : ")
    print(FP/float(TN+FP))    

    print("Testing Accuracy : ", metrics.accuracy_score(y_test, y_pred))
    return metrics.accuracy_score(y_test, y_pred)

###  Strip the input data

In [6]:
train = pd.read_csv("train.csv", index_col=0)
test = pd.read_csv("test.csv", index_col=0)
print(train.shape, test.shape)

(891, 11) (418, 10)


In [7]:
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
cols_to_drop = ["Name", "Ticket", "Cabin"]

In [9]:
train.drop(cols_to_drop, axis= 1, inplace=True)
test.drop(cols_to_drop, axis= 1, inplace=True)

In [10]:
gender_mapping = {'female':0, 'male':1}
train['Sex'] = train['Sex'].map(gender_mapping)
test['Sex'] = test['Sex'].map(gender_mapping)

In [11]:
embarked_mapping = {'S':0, 'C':1, 'Q': 2}
train['Embarked'] = train['Embarked'].map(embarked_mapping)
test['Embarked'] = test['Embarked'].map(embarked_mapping)

In [12]:
test.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000,418.000000
mean,2.265550,0.636364,30.272590,0.447368,0.392344,35.627188,0.464115
std,0.841838,0.481622,14.181209,0.896760,0.981429,55.907576,0.685516
min,1.000000,0.000000,0.170000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,21.000000,0.000000,0.000000,7.895800,0.000000
50%,3.000000,1.000000,27.000000,0.000000,0.000000,14.454200,0.000000
75%,3.000000,1.000000,39.000000,1.000000,0.000000,31.500000,1.000000
max,3.000000,1.000000,76.000000,8.000000,9.000000,512.329200,2.000000


### Dealing with missing values

In [13]:
test.Age.fillna(inplace=True, value=test.Age.mean())
test.Fare.fillna(inplace=True, value=test.Fare.mean())

In [14]:
test.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,2.265550,0.636364,30.272590,0.447368,0.392344,35.627188,0.464115
std,0.841838,0.481622,12.634534,0.896760,0.981429,55.840500,0.685516
min,1.000000,0.000000,0.170000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,23.000000,0.000000,0.000000,7.895800,0.000000
50%,3.000000,1.000000,30.272590,0.000000,0.000000,14.454200,0.000000
75%,3.000000,1.000000,35.750000,1.000000,0.000000,31.500000,1.000000
max,3.000000,1.000000,76.000000,8.000000,9.000000,512.329200,2.000000


In [15]:
train.dropna(inplace=True)
test.dropna(inplace=True)
print(train.shape, test.shape)

(712, 8) (418, 7)


In [16]:
y = train.Survived
train.drop("Survived", axis= 1, inplace=True)
X = train
X_final = test

### Logisitic regression evaluation

In [17]:
logreg = LogisticRegression()
print(report_train_test_accuracy(logreg, X, y, 0.4, 9))

Confusion Matrix : 
[[153  25]
 [ 34  73]]
Sensitivity :
0.682242990654
Precision : 
0.744897959184
Specificity : 
0.859550561798
False Positive Rate : 
0.140449438202
Testing Accuracy :  0.79298245614
0.79298245614


In [18]:
logreg = LogisticRegression()
print(report_train_accuracy(logreg, X, y))

Confusion Matrix : 
[[369  55]
 [ 91 197]]
Sensitivity :
0.684027777778
Precision : 
0.781746031746
Specificity : 
0.870283018868
False Positive Rate : 
0.129716981132
Training Accuracy :  0.794943820225
0.794943820225


In [19]:
logreg = LogisticRegression()
print(cross_val_score(logreg, X, y, cv=10, scoring='accuracy').mean())

0.788054437738


### KNN evaluation

In [21]:
knn = KNeighborsClassifier(n_neighbors=5, weights='distance')
print(report_train_accuracy(knn, X, y))

Confusion Matrix : 
[[424   0]
 [ 10 278]]
Sensitivity :
0.965277777778
Precision : 
1.0
Specificity : 
1.0
False Positive Rate : 
0.0
Training Accuracy :  0.98595505618
0.98595505618


In [22]:
knn = KNeighborsClassifier(n_neighbors=5, weights='distance')
print(report_train_test_accuracy(knn, X, y, 0.4, 9))

Confusion Matrix : 
[[134  44]
 [ 46  61]]
Sensitivity :
0.570093457944
Precision : 
0.580952380952
Specificity : 
0.752808988764
False Positive Rate : 
0.247191011236
Testing Accuracy :  0.684210526316
0.684210526316


In [23]:
knn = KNeighborsClassifier(n_neighbors=5, weights='distance')
print(cross_val_score(knn, X, y, cv=10, scoring='accuracy').mean())

0.685620389001


###  Use Decision Tree

In [24]:
tree1 = tree.DecisionTreeClassifier()
print(report_train_accuracy(tree1, X, y))

Confusion Matrix : 
[[424   0]
 [ 10 278]]
Sensitivity :
0.965277777778
Precision : 
1.0
Specificity : 
1.0
False Positive Rate : 
0.0
Training Accuracy :  0.98595505618
0.98595505618


In [27]:
tree1 = tree.DecisionTreeClassifier()
print(report_train_test_accuracy(tree1, X, y, 0.4, 9))

Confusion Matrix : 
[[128  50]
 [ 32  75]]
Sensitivity :
0.700934579439
Precision : 
0.6
Specificity : 
0.719101123596
False Positive Rate : 
0.280898876404
Testing Accuracy :  0.712280701754
0.712280701754


In [28]:
tree1 = tree.DecisionTreeClassifier()
print(cross_val_score(tree1, X, y, cv=10, scoring='accuracy').mean())

0.768489268947
